# Deep Neural Networks Marathon

In [1]:
# import lib
import torch

In [2]:
# torch version
print('torch version',torch.__version__)
# check gpu
print('number of gpu',torch.cuda.device_count())
for id in range(torch.cuda.device_count()):
    gpu = torch.cuda.get_device_properties(id)
    print('gpu info',gpu)

torch version 2.1.0+cu121
number of gpu 1
gpu info _CudaDeviceProperties(name='NVIDIA GeForce RTX 3050 4GB Laptop GPU', major=8, minor=6, total_memory=4095MB, multi_processor_count=20)


In [21]:
#alexnet

import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.pool4(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [31]:
model = AlexNet(num_classes=10)
print(model)

AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=10, bias=True)
)


In [30]:
from torchsummary import summary

summary(model=AlexNet(), input_size=(3,224,224), batch_size=1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 96, 55, 55]          34,944
         MaxPool2d-2            [1, 96, 27, 27]               0
            Conv2d-3           [1, 256, 27, 27]         614,656
         MaxPool2d-4           [1, 256, 13, 13]               0
            Conv2d-5           [1, 384, 13, 13]         885,120
            Conv2d-6           [1, 256, 13, 13]         884,992
         MaxPool2d-7             [1, 256, 6, 6]               0
            Linear-8                  [1, 4096]      37,752,832
            Linear-9                  [1, 4096]      16,781,312
           Linear-10                    [1, 10]          40,970
Total params: 56,994,826
Trainable params: 56,994,826
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 5.46
Params size (MB): 217.42
Est

In [ ]:
# load data
import torch

# 定义数据集类
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.data)

# 加载数据
data = torch.randn(100, 10)
labels = torch.randint(0, 10, (100,))

# 创建数据集
dataset = MyDataset(data, labels)

# 加载数据
loader = torch.utils.data.DataLoader(dataset, batch_size=128)

# 迭代加载器
for images, labels in loader:
    print(images.size())
    print(labels.size())
